# Load libraries and data

In [1]:
### Load libraries 
import os
import numpy as np
import pandas as pd
import qgrid
import enum
from datetime import datetime

from traitlets import All
from collections import OrderedDict

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from ipywidgets import *
from IPython.display import clear_output, display
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

class TABS_PRIMARY(enum.Enum):
    Criteria = 0
    Select = 1
    Filter = 2
    Apply = 3
    Undo = 4
    
class TABS_SAVE(enum.Enum):
    Type = 0
    New = 1
    Existing = 2
    Save = 3

import helpers.research_helpers as app    

OPTIONS_CSV_COLUMNS = []
#OPTIONS_EXISTING_PROJECTS = app.load_existing_projects([])
#OPTIONS_EXISTING_PROJECTS = []
OPTIONS_EXISTING_PROJECTS_RESEARCH = []
OPTIONS_EXISTING_PROJECTS_MARKET_RESEARCH = []
#OPTIONS_EXISTING_PROJECTS_RESEARCH = app.load_existing_projects([])
#OPTIONS_EXISTING_PROJECTS_MARKET_RESEARCH = app.load_existing_projects([], True)
OPTIONS_LOCATIONS = []

dict_qgrid = {}     # global variable to store the qgrid in the "Filter by" tab

print('Notebook initialized.')

Notebook initialized.


In [2]:
### Load most recent, cleaned and transformed membership list from '../sampling/Member Lists/all_active.csv' 

df_complete_list = app.load_csv(os.path.join(app.FLDR_SAMPLING, app.FLDR_MEMBER_LISTS, app.FNAME_ACTIVE_ROSTER_CSV))
app.dict_dataframes['Initial dataset'] = df_complete_list.copy()

print(str(len(df_complete_list)) + ' records loaded from ' + os.path.join(app.FLDR_SAMPLING, app.FLDR_MEMBER_LISTS, app.FNAME_ACTIVE_ROSTER_CSV))

303102 records loaded from /media/Sampling/Member Lists/all_active.csv


In [3]:
### Define additional variables which are dependant on the dataset being loaded first 

OPTIONS_CSV_COLUMNS = np.sort(np.array(df_complete_list.columns.unique().tolist()))
OPTIONS_LOCATIONS = np.sort(np.array(df_complete_list[app.COL_LOCATION_CALCULATED].unique().tolist()))

# Remove Samples & Filter by Categorical Variables

In [4]:
# Required Functions 

#def __remove_prev_samples(df_left, base_path, folder_name, filename): 
def __remove_prev_samples(df_left, full_path, filename): 
    # 1. load file into df
    df_sample = pd.read_csv (os.path.join(full_path, filename) , sep=app.CSV_SEPARATOR, header='infer')
    # 2. convert column names to lower case
    df_sample.columns=df_sample.columns.str.lower()
    # 3. rename variations of 'email' to simply 'email'
    df_sample.rename(columns={"email_addr": "email", "email_address": "email", "emailaddr": "email"}, inplace = True)
    # 3.x drop all other colums
    df_temp = pd.DataFrame(columns=['email'])
    df_temp['email'] = df_sample['email']
    # 3.x add new column called 'prev_sampled' and set value to 1
    df_temp['prev_sample'] = 1
    # 3.x pd.merge(left=df_left, right=df_right, how='left', left_on='email', right_on='email')
    df_left = pd.merge(left=df_left, right=df_temp, how='left', left_on='email', right_on='email')
    # 3.x drop, or filter out, where 'prev_sampled' == 1
    df_left['prev_sample'] = df_left['prev_sample'].fillna(0)
    # 4 go back to step and repeat all of step 3 using member_id (IF IT EXISTS)
    df_left = df_left[df_left['prev_sample'] == 0]
    df_left = df_left.drop(columns=['prev_sample'])
    #print(str(len(df_left))) 
    return df_left

def remove_prev_samples(df_complete_list, df_previous_sample_files, dropdown_value, column_name):
    count_orig = len(df_complete_list)

    for index, row in df_previous_sample_files.iterrows():
        dropdown_research_type = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[1]
        if dropdown_research_type.value == 'General Research':
            full_path = os.path.join(app.FLDR_SAMPLING, dropdown_value)
        else:
            full_path = os.path.join(app.FLDR_MARKET_RESEARCH, dropdown_value, '4_Recruitment', 'Sample list')

        
        df_complete_list = __remove_prev_samples(df_complete_list, full_path, row[column_name])

    count_new = len(df_complete_list)
    print('You started with {:,} possible members.  We removed {:,}.  You have {:,} remaining.'.format(count_orig, (count_orig - count_new), count_new))
    
    return df_complete_list

def revert_dropdown__get_options_from_dict(dict):
    keys = []
    for key in dict:
        key_text = '{}'.format(key)
        key_length = '{:,}'.format(len(dict[key]))
        key_length = '{: >7}'.format(key_length)
        keys.append('{} | {}'.format(key_length, key_text))
    return keys

def __revert_btn__revert_dict_state(revert_key, dict):
    dict_keys_to_remove = {}
    bFoundKey = False
    
    for key in dict:
        if bFoundKey:
            dict_keys_to_remove[key] = 1
            
        if key == revert_key:
            bFoundKey = True    

    for key in dict_keys_to_remove:
        removed_value = dict.pop(key)
        print('Removed: {}'.format(key))

    return dict

def tabstrip_primary__dropdown_criteria_changed(widget):
    dropdown_criteria = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[0]
    dropdown_research_type = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[1]
    
    if widget['new'] == 'Remove previous samples':
        dropdown_research_type.layout.visibility = 'visible'
    else:
        dropdown_research_type.layout.visibility = 'hidden'

def tabstrip_primary__revert_btn_clicked(btn):
    with tabstrip_primary.children[TABS_PRIMARY.Undo.value].children[1]:
        clear_output()
    
        if len(app.dict_dataframes) > 1:
            dropdown = tabstrip_primary.children[TABS_PRIMARY.Undo.value].children[0].children[0]
            app.dict_dataframes = __revert_btn__revert_dict_state(dropdown.value.split(' | ')[-1], app.dict_dataframes)
            dropdown.options = revert_dropdown__get_options_from_dict(app.dict_dataframes)
            return_value = 'Finished reverting values.'
        else:
            return_value = 'Nothing to do.'
    
        print(return_value)
    
    return return_value

def tabstrip_primary__on_tab_click(widget):
    COL_NEW_EXISTING_FILENAME = 'filename'
    tab_idx_prev = widget['old']
    tab_idx = widget['new']
    
    output_widget = tabstrip_primary.children[tab_idx].children[-1]
    with output_widget:
        if tab_idx != TABS_PRIMARY.Criteria.value:
            dropdown_criteria = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[0]
        
        if tab_idx == TABS_PRIMARY.Select.value:
            clear_output()
            
            #tabstrip_primary.children[TABS_PRIMARY.Select.value].children[0].children[0].options = []
            
            if dropdown_criteria.value == 'Remove previous samples':
                dropdown_research_type = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[1]
                
                if dropdown_research_type.value == 'General Research':
                    option_array = app.load_existing_projects([], True)
                else:
                    option_array = app.load_existing_projects([], False)
            else:
                option_array = OPTIONS_CSV_COLUMNS
            
            
            dropdown = tabstrip_primary.children[TABS_PRIMARY.Select.value].children[0].children[0].options = option_array
            
        if tab_idx == TABS_PRIMARY.Filter.value:
            clear_output()
            if tab_idx_prev == TABS_PRIMARY.Select.value:
                #print('running long function')
                #display(tabstrip_primary.children[0].children[0].value)
                dropdown = tabstrip_primary.children[TABS_PRIMARY.Select.value].children[0].children[0]
                if dropdown_criteria.value == 'Remove previous samples':
                    app.COL_KEEP = 'remove'
                    KEEP_LIST_FILES = np.array([])
                    df_new_existing = pd.DataFrame(columns=[COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID])
                    
                    dropdown_research_type = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[1]
                    if dropdown_research_type.value == 'General Research':
                        path_to_csvs = os.path.join(app.FLDR_SAMPLING, dropdown.value)
                        
                    else:
                        path_to_csvs = os.path.join(app.FLDR_MARKET_RESEARCH, dropdown.value, '4_Recruitment', 'Sample list')
                    
                    if dropdown_research_type.value == 'General Research' and dropdown.value == '<By Sample Date>':
                        ARRAY_SAMPLE_FILES = []
                        ndx = 0
                        def scanRecurse(baseDir):
                            
                            df_temp = pd.DataFrame(columns=[COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID])
                            for entry in os.scandir(baseDir):
                                if entry.is_file():
                                    if entry.name[-4:] == '.csv':
                                        if 'forQualtrics' in entry.name:
                                            if 'EST_' not in entry.name and 'CST_' not in entry.name and 'MST_' not in entry.name and 'PST_' not in entry.name:
                                                ARRAY_SAMPLE_FILES.append(entry.name[-14:-4] + ' | ' + os.path.join(baseDir, entry.name))
                                                #print(os.path.join(entry.name[-14:-4] + ' | ' + entry.name))
                                                #print(os.path.join(baseDir, entry.name[-14:-4] + ' | ' + entry.name))
                                else:
                                    if entry.is_dir(follow_symlinks=False):
                                        if entry.path.split('/')[-1] not in app.ARRAY_FOLDERS_TO_IGNORE:
                                            scanRecurse(entry.path)

                        scanRecurse(app.FLDR_SAMPLING)
                        
                        ARRAY_SAMPLE_FILES.sort(reverse=True)

                        for filename in ARRAY_SAMPLE_FILES:
                            #print(filename)
                            shortend_filename = filename.replace(app.FLDR_SAMPLING, '')[14:]
                            #2021-08-12 | /media/Sampling/
                            #print(shortend_filename)
                            new_row = {COL_NEW_EXISTING_FILENAME:shortend_filename, app.COL_MEMBER_ID:ndx}
                            df_new_existing = df_new_existing.append(new_row, ignore_index=True)
                            delta = datetime.now()-datetime.strptime(shortend_filename[-14:-4], '%Y-%m-%d')
                            if delta.days <= 30:
                                KEEP_LIST_FILES = np.append(KEEP_LIST_FILES, shortend_filename)
                                #print('{}, {}'.format(delta.days, filename))
                        
                        #print(df_new_existing.head())
                    else:
                        dirs = os.scandir(path = path_to_csvs)
                        ndx = 0
                        for entry in dirs :
                            #ndx = ndx + 1
                            if entry.is_file() and entry.name[-4:] == '.csv':
                                if 'forQualtrics' in entry.name:
                                    KEEP_LIST_FILES = np.append(KEEP_LIST_FILES, entry.name)
                                    #KEEP_LIST_FILES.append(entry.name)

                                new_row = {COL_NEW_EXISTING_FILENAME:entry.name, app.COL_MEMBER_ID:ndx}
                                df_new_existing = df_new_existing.append(new_row, ignore_index=True)

                    app.dict_filter_by['remove_samples_input_df'] = df_new_existing
                    dict_qgrid['primary'] = app.keep_grid_show_filter(df_new_existing, COL_NEW_EXISTING_FILENAME, app.COL_MEMBER_ID, KEEP_LIST_FILES)
                else:
                    print(dropdown.value)
                    app.COL_KEEP = 'keep'
                    #print(app.dict_dataframes[list(app.dict_dataframes)[-1]])
                    dict_qgrid['primary'] = app.keep_grid_show_filter(app.dict_dataframes[list(app.dict_dataframes)[-1]], dropdown.value, app.COL_MEMBER_ID, app.get_keep_list(dropdown.value))
            else:
                print('You cannot complete this step until you have completed step {}'.format(TABS_PRIMARY.Select.value))
            
        if tab_idx == TABS_PRIMARY.Apply.value:
            clear_output()
            
            if tab_idx_prev == TABS_PRIMARY.Filter.value and 'primary' in dict_qgrid.keys():
                dropdown = tabstrip_primary.children[TABS_PRIMARY.Select.value].children[0].children[0]
                dropdown_research_type = tabstrip_primary.children[TABS_PRIMARY.Criteria.value].children[0].children[1]
                if dropdown_criteria.value == 'Remove previous samples':
                    
                    dropdown_value = dropdown.value
                    if dropdown_research_type.value == 'General Research' and dropdown.value == '<By Sample Date>':
                        dropdown_value = ''
                        
                    df_previous_sample_files, app.dict_keep_lists[dropdown.value] = app.keep_grid_apply_filter(dict_qgrid['primary'], COL_NEW_EXISTING_FILENAME, app.dict_filter_by['remove_samples_input_df'], True)
                    print('Please wait while we remove the previous samples from the current dataset...')
                    df_filtered_list = remove_prev_samples(app.dict_dataframes[list(app.dict_dataframes)[-1]], df_previous_sample_files, dropdown_value, COL_NEW_EXISTING_FILENAME)
                    #df_filtered_list = remove_prev_samples(app.dict_dataframes['Initial list'], df_previous_sample_files, dropdown.value)
                    app.dict_dataframes[dropdown_criteria.value + ' from ' + dropdown.value] = df_filtered_list.copy()
                else:
                    df_filtered_list, app.dict_keep_lists[dropdown.value] = app.keep_grid_apply_filter(dict_qgrid['primary'], dropdown.value, app.dict_dataframes[list(app.dict_dataframes)[-1]], True)
                    app.dict_dataframes[dropdown_criteria.value + ' - ' + dropdown.value] = df_filtered_list.copy()
                
                dict_qgrid.pop('primary', None)
            else:
                print('You cannot complete this step until you have completed step {}'.format(TABS_PRIMARY.Filter.value))
        if tab_idx == TABS_PRIMARY.Undo.value:
            clear_output()
            
            tabstrip_primary.children[tab_idx].children[0].children[0].options = revert_dropdown__get_options_from_dict(app.dict_dataframes)

def tabstrip_primary__setup():            
    outputs = {i: widgets.VBox() for i in range(0,5)}

    #def dropdown_criteria_eventhandler(change):
    #    print(change.new)
    #
    #dropdown_criteria.observe(dropdown_criteria_eventhandler, names='value')
    dropdown_criteria = app.build_dropdown(['Remove previous samples', 'Filter on categorical variables'], None, 'I want to...')
    dropdown_criteria.observe(tabstrip_primary__dropdown_criteria_changed, 'value')
    dropdown_research_type = app.build_dropdown(['General Research', 'Market Research'], None, 'Project type')
    outputs[TABS_PRIMARY.Criteria.value].children = [widgets.VBox(children=[dropdown_criteria, dropdown_research_type]), widgets.Output()]
    outputs[TABS_PRIMARY.Select.value].children = [widgets.VBox(children=[app.build_dropdown([], None, '')]), widgets.Output()]
    outputs[TABS_PRIMARY.Filter.value].children = [widgets.VBox(), widgets.Output()]
    outputs[TABS_PRIMARY.Apply.value].children = [widgets.VBox(), widgets.Output()]

    revert_btn = widgets.Button(value="revert", description="Revert to Previous State", layout=widgets.Layout(width="500px"))
    revert_btn.on_click(tabstrip_primary__revert_btn_clicked)
    outputs[TABS_PRIMARY.Undo.value].children = [widgets.HBox(children=[app.build_dropdown([], None, ''), revert_btn]), widgets.Output()]

    # add the Output widgets as tab childen
    tabstrip_primary = widgets.Tab()
    tabstrip_primary.children = list(outputs.values())

    for i, title in outputs.items():
        if i == TABS_PRIMARY.Criteria.value:
            tabstrip_primary.set_title(i, 'Step 0 - Criteria')
        if i == TABS_PRIMARY.Select.value:
            tabstrip_primary.set_title(i, 'Step 1 - Select')
        if i == TABS_PRIMARY.Filter.value:
            tabstrip_primary.set_title(i, 'Step 2 - Filter')
        if i == TABS_PRIMARY.Apply.value:
            tabstrip_primary.set_title(i, 'Step 3 - Apply')
        if i == TABS_PRIMARY.Undo.value:
            tabstrip_primary.set_title(i, 'View / Undo')

    tabstrip_primary.observe(tabstrip_primary__on_tab_click, names='selected_index')
    
    return tabstrip_primary

tabstrip_primary = tabstrip_primary__setup()
display(tabstrip_primary)

# Save output

In [5]:
# Required Functions 

## Informational only - How to calculate sample_size 
#seed_value = 4500
#resp_rate = .03
#target_complete = 1000
#sample_size = round(target_complete / resp_rate if (target_complete / resp_rate) < len(df_filtered_list) else len(df_filtered_list))

def print_info_msg(msg):
    info_lbl = widgets.HTML()
    info_lbl.value = '<span style="font-size:12pt">{}</span>'.format(msg)
    return info_lbl

def print_error_msg(msg):
    error_lbl = widgets.HTML()
    error_lbl.value = '<span style="font-size:12pt"><font color="red"><b>Error: </b></font>{}</span>'.format(msg)
    return error_lbl

def __get_date(): 
    now = datetime.now()
    return now.strftime("%Y-%m-%d")

def tabstrip_save__save_btn_clicked(btn):
    seed_value = tabstrip_save.children[TABS_SAVE.Save.value].children[0].children[0].value
    sample_size = tabstrip_save.children[TABS_SAVE.Save.value].children[0].children[1].value
    max_sample_size = len(app.dict_dataframes[list(app.dict_dataframes)[-1]])
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=10,
        description='Saving:',
        bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
        style={'bar_color': 'maroon'},
        orientation='horizontal'
    )
    with tabstrip_save.children[TABS_SAVE.Save.value].children[1]:     # widget.output() for the progress bar
        display(progress_bar)
    
    with tabstrip_save.children[TABS_SAVE.Save.value].children[2]:     # widget.output() for messages
        clear_output()
        
        if sample_size == 0 or sample_size > max_sample_size:
            display(print_error_msg('Please provide a sample size between 1 and {:,}'.format(max_sample_size)))
            return
            
        clear_output()
        display(print_info_msg('Setting up variables'))
        date_string = __get_date()
        project_name = tabstrip_save.children[TABS_SAVE.Existing.value].children[0].children[0]
        
        dropdown_research_type = tabstrip_save.children[TABS_SAVE.Type.value].children[0].children[0]
        if dropdown_research_type.value == 'General Research':
            new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name.value)

        else:
            new_abs_path = os.path.join(os.getcwd(), app.FLDR_MARKET_RESEARCH, project_name.value, '4_Recruitment', 'Sample list')
            if not os.path.exists(new_abs_path):
                try:
                    os.makedirs(new_abs_path, exist_ok = True)
                except OSError as error:
                    print("Directory '%s' can not be created")
        
        #new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name.value)
        progress_bar.value += 1

        clear_output()
        display(print_info_msg('/all_active_{}.csv'.format(date_string)))
        df_complete_list = app.dict_dataframes[list(app.dict_dataframes)[0]]
        df_complete_list.to_csv(new_abs_path + '/all_active_{}.csv'.format(date_string), header=True, index=False)
        progress_bar.value += 1

        clear_output()
        display(print_info_msg('/all_active_filtered_{}.csv'.format(date_string)))
        df_filtered_list = app.dict_dataframes[list(app.dict_dataframes)[-1]]
        df_filtered_list.to_csv(new_abs_path + '/all_active_filtered_{}.csv'.format(date_string), header=True, index=False)
        progress_bar.value += 1

        clear_output()
        display(print_info_msg('/all_active_remaining_{}.csv'.format(date_string)))
        df_remaining = df_filtered_list.loc[~df_filtered_list[app.COL_MEMBER_ID].isin(df_complete_list[app.COL_MEMBER_ID])].copy()
        df_remaining.to_csv(new_abs_path + '/all_active_remaining_{}.csv'.format(date_string), header=True, index=False)
        progress_bar.value += 1

        clear_output()
        display(print_info_msg('/full_sample_{}.csv'.format(date_string)))
        df_sample = df_filtered_list.sample(n = sample_size, replace = False, random_state = seed_value)
        df_sample.to_csv(new_abs_path + '/full_sample_{}.csv'.format(date_string), header=True, index=False)
        progress_bar.value += 1

        clear_output()
        display(print_info_msg('/full_sample_N{}_forQualtrics_complete_{}.csv'.format(sample_size, date_string)))
        df_qualtrics = df_sample[[app.COL_MEMBER_ID, 'first_name', 'last_name', app.COL_EMAIL]]
        df_qualtrics.to_csv(new_abs_path + '/full_sample_N{}_forQualtrics_complete_{}.csv'.format(sample_size, date_string), header=True, index=False)
        progress_bar.value += 1

        time_zone = "EST"
        clear_output()
        display(print_info_msg('/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string)))
        df_tz = df_sample[df_sample['time_zone_calculated'] == time_zone]
        df_qualtrics = df_tz[[app.COL_MEMBER_ID, 'first_name', 'last_name', app.COL_EMAIL]] 
        df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)
        progress_bar.value += 1

        time_zone = "CST"
        clear_output()
        display(print_info_msg('/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string)))
        df_tz = df_sample[df_sample['time_zone_calculated'] == time_zone]
        df_qualtrics = df_tz[[app.COL_MEMBER_ID, 'first_name', 'last_name', app.COL_EMAIL]] 
        df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)
        progress_bar.value += 1

        time_zone = "MST"
        clear_output()
        display(print_info_msg('/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string)))
        df_tz = df_sample[df_sample['time_zone_calculated'] == time_zone]
        df_qualtrics = df_tz[[app.COL_MEMBER_ID, 'first_name', 'last_name', app.COL_EMAIL]] 
        df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)
        progress_bar.value += 1

        time_zone = "PST"
        clear_output()
        display(print_info_msg('/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string)))
        df_tz = df_sample[df_sample['time_zone_calculated'] == time_zone]
        df_qualtrics = df_tz[[app.COL_MEMBER_ID, 'first_name', 'last_name', app.COL_EMAIL]] 
        df_qualtrics.to_csv(new_abs_path + '/sample_{}_N{}_forQualtrics_{}.csv'.format(time_zone, sample_size, date_string), header=True, index=False)
        progress_bar.value += 1
        progress_bar.description = 'Finished'
        clear_output()

def tabstrip_save__verify_btn_clicked(btn):
    with tabstrip_save.children[TABS_SAVE.New.value].children[1]:
        clear_output()
        
        project_name = tabstrip_save.children[TABS_SAVE.New.value].children[0].children[0].value
            
        if project_name != '':
            dropdown_research_type = tabstrip_save.children[TABS_SAVE.Type.value].children[0].children[0]
            
            if dropdown_research_type.value == 'General Research':
                new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name)
            else:
                new_abs_path = os.path.join(os.getcwd(), app.FLDR_MARKET_RESEARCH, project_name)
            
            if os.path.exists(new_abs_path):
                print('The project name ({}) already exists.'.format(project_name))
                print('Please specify a different project name.')
                tabstrip_save.children[TABS_SAVE.New.value].children[0].children[0].value = ''
                btn.value = ''
            else:
                print('Created project folder in: \n{}'.format(new_abs_path))
                os.mkdir(new_abs_path)
                
                if dropdown_research_type.value == 'General Research':
                    option_array = app.load_existing_projects([], True)
                else:
                    option_array = app.load_existing_projects([], False)
                    
                tabstrip_save.children[TABS_SAVE.Existing.value].children[0].children[0].options = option_array
                tabstrip_save.children[TABS_SAVE.Existing.value].children[0].children[0].value = project_name
                btn.value = 'Verified'
        else:
            print('Please provide a project name before clicking verify.')


def tabstrip_save__on_tab_click(widget):
    tab_idx_prev = widget['old']
    tab_idx = widget['new']
    
    output_widget = tabstrip_save.children[tab_idx].children[1]
        
    with output_widget:
        if tab_idx == TABS_SAVE.Type.value:
            clear_output()
        
        if tab_idx == TABS_SAVE.New.value:
            clear_output()
            
        if tab_idx == TABS_SAVE.Existing.value:
            clear_output()
            
            project_name = tabstrip_save.children[TABS_SAVE.Existing.value].children[0].children[0].value
            dropdown_research_type = tabstrip_save.children[TABS_SAVE.Type.value].children[0].children[0]
            
            if dropdown_research_type.value == 'General Research':
                new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name)
            else:
                new_abs_path = os.path.join(os.getcwd(), app.FLDR_MARKET_RESEARCH, project_name)
                
        if tab_idx == TABS_SAVE.Save.value:
            clear_output()
            
            max_sample_size = len(app.dict_dataframes[list(app.dict_dataframes)[-1]])
            tabstrip_save.children[TABS_SAVE.Save.value].children[0].children[1].max = max_sample_size
            
            project_name = tabstrip_save.children[TABS_SAVE.Existing.value].children[0].children[0].value

            if project_name == '':
                with tabstrip_save.children[tab_idx].children[2]:
                    display(print_error_msg('Please provide a project name before trying to save.'))
            else:
                dropdown_research_type = tabstrip_save.children[TABS_SAVE.Type.value].children[0].children[0]
                
                if dropdown_research_type.value == 'General Research':
                    new_abs_path = os.path.join(os.getcwd(), app.FLDR_SAMPLING, project_name)
                else:
                    new_abs_path = os.path.join(os.getcwd(), app.FLDR_MARKET_RESEARCH, project_name)
                
                display(print_info_msg('Your work will be saved in: <b>{}</b>'.format(new_abs_path)))
                display(print_info_msg('Max Sample Size = <b>{:,}</b>'.format(len(app.dict_dataframes[list(app.dict_dataframes)[-1]]))))
                
def tabstrip_save__dropdown_research_type_changed(widget): 
    dropdown_research_type = tabstrip_save.children[TABS_SAVE.Type.value].children[0].children[0]
    
    if dropdown_research_type.value == 'General Research':
        option_array = app.load_existing_projects([], True)
    else:
        option_array = app.load_existing_projects([], False)
        
    tabstrip_save.children[TABS_SAVE.Existing.value].children[0].children[0].options = option_array

def tabstrip_save__setup(): 
    outputs = {i: widgets.VBox() for i in range(0,4)}

    dropdown_research_type = app.build_dropdown(['General Research', 'Market Research'], None, '')
    dropdown_research_type.observe(tabstrip_save__dropdown_research_type_changed, 'value')
    outputs[TABS_SAVE.Type.value].children = [widgets.VBox(children=[dropdown_research_type]), widgets.Output()]
    
    project_name = widgets.Text(value='', placeholder='Project Name', description='', disabled=False, width='500px')
    verify_btn = widgets.Button(value='', description="Verify Name & Create Project Directory", layout=widgets.Layout(width="300px"))
    verify_btn.on_click(tabstrip_save__verify_btn_clicked)
    outputs[TABS_SAVE.New.value].children = [widgets.VBox(children=[project_name, verify_btn, widgets.Label(value="If you are creating a new project, please enter the name above and verify it does NOT already exist.")]), widgets.Output()]
    
    outputs[TABS_SAVE.Existing.value].children = [widgets.VBox(children=[app.build_dropdown(app.load_existing_projects([]), None, '')]), widgets.Output()]
    
    seed_value = widgets.BoundedIntText(value=4500, min=0, max=10000, step=1, description='Seed Value:', disabled=False)
    sample_size = widgets.BoundedIntText(value=100, min=1, max=1000000, step=1, description='Sample Size:', disabled=False)
    save_btn = widgets.Button(value="", description="Save", disabled=False, layout=widgets.Layout(width="300px"))
    save_btn.on_click(tabstrip_save__save_btn_clicked)
    
    outputs[TABS_SAVE.Save.value].children = [widgets.VBox(children=[seed_value, sample_size, save_btn]), widgets.Output(), widgets.Output()]

    tabstrip_save = widgets.Tab()
    tabstrip_save.children = list(outputs.values())

    for i, title in outputs.items():
        if i == TABS_SAVE.Type.value:
            tabstrip_save.set_title(i, 'GR or MR')
        if i == TABS_SAVE.New.value:
            tabstrip_save.set_title(i, 'Create Project, or')
        if i == TABS_SAVE.Existing.value:
            tabstrip_save.set_title(i, 'Select Project, then')
        if i == TABS_SAVE.Save.value:
            tabstrip_save.set_title(i, 'Save')

    tabstrip_save.observe(tabstrip_save__on_tab_click, names='selected_index')
    
    return tabstrip_save

tabstrip_save = tabstrip_save__setup()
display(tabstrip_save)